`Module 3`
# Setting up a RAG pipeline using Cortex Search

## Install prerequisite library

Click on *"Packages"* in the top-right corner and enter `snowflake-ml-python` and click the *"Save"* button.

## Query Customer Reviews Data with Sentiment Score

In [ ]:
SELECT * FROM AVALANCHE_DB.AVALANCHE_SCHEMA.REVIEWS_WITH_SENTIMENT LIMIT 5

## Prepare Review Text for Analysis (Chunking)

In [ ]:
CREATE OR REPLACE TABLE AVALANCHE_DB.AVALANCHE_SCHEMA.CHUNKED_CONTENT (
    file_name VARCHAR,
    CHUNK VARCHAR
);

INSERT INTO AVALANCHE_DB.AVALANCHE_SCHEMA.CHUNKED_CONTENT (file_name, CHUNK)
SELECT
    FILENAME,
    c.value AS CHUNK
FROM
    AVALANCHE_DB.AVALANCHE_SCHEMA.REVIEWS_WITH_SENTIMENT,
    LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
        REVIEW_TEXT,
        'markdown',
        1800,
        250
    )) c;

## Query Chunked Content

In [ ]:
SELECT * FROM AVALANCHE_DB.AVALANCHE_SCHEMA.CHUNKED_CONTENT LIMIT 5

## Create Cortex Search Service

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE AVALANCHE_DB.AVALANCHE_SCHEMA.AVALANCHE_SEARCH_SERVICE
    ON chunk
    WAREHOUSE = compute_wh
    TARGET_LAG = '1 minute'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
    SELECT
        file_name,
        chunk
    FROM AVALANCHE_DB.AVALANCHE_SCHEMA.CHUNKED_CONTENT
    );

## Query the Cortex Search Service (SQL)

In [ ]:
-- Query it with SQL
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'AVALANCHE_DB.AVALANCHE_SCHEMA.AVALANCHE_SEARCH_SERVICE',
      '{
         "query": "Any goggles review?",
         "columns":[
            "file_name",
            "CHUNK"
         ],
         "limit":3
      }'
  )
)['results'] as results;

## Query the Cortex Search Service (Python)

In [ ]:
# Query it with Python
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session
import streamlit as st
import json
import pandas as pd

session = get_active_session()

prompt="Any goggles review?"

root = Root(session)

# Query service
svc = (root
  .databases["AVALANCHE_DB"]
  .schemas["AVALANCHE_SCHEMA"]
  .cortex_search_services["AVALANCHE_SEARCH_SERVICE"]
)

resp = svc.search(
  query=prompt,
  columns=["CHUNK", "file_name"],
  limit=3
).to_json()

# JSON formatting
json_conv = json.loads(resp) if isinstance(resp, str) else resp
search_df = pd.json_normalize(json_conv['results'])

for _, row in search_df.iterrows():
    st.write(f"**{row['CHUNK']}**")
    st.caption(row['file_name'])
    st.write('---')